# Bay Area - County Salary Comparison


### Introduction
The motivation of the notebook is to compare graduate student assistant salaries across various counties and compare them to the state's salaries. For the purpose of this study, only data from public records were used which can be found on the web under many names e.g. "Civic Service Pay Scale", "Job Salary Schedule", etc.

**Note**: These salaries are *not* to be confused with "New Graduate Salaries", rather these salaries are "Graduate Student internship" levels.  
<br />

<center><img src="https://upload.wikimedia.org/wikipedia/commons/d/d8/California_Bay_Area_county_map_%28zoom%26color%29.svg" alt="bay area counties" width="250" height="250"></center>


**Hypothesis**: Counties of interest near Silicon Valley are at or exceed state salaries.
* San Francisco, San Mateo, Santa Clara, Alameada
* Santa Cruz which is south of San Mateo (grayed out on the map) was added out of interest